<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 1 | Question 2
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Load and Prepare Dataset

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
replace hw1-files/q3/patches.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hw1-files/q3/patches.csv  A
A

  inflating: hw1-files/q3/lsh.py     
  inflating: hw1-files/q1/dataset1.txt  
  inflating: hw1-files/q2/games_library.txt  


In [2]:
import pandas as pd
dataset = pd.read_csv("/content/hw1-files/q2/games_library.txt", header=None)

In [3]:
dataset[0] = dataset[0].apply(lambda x: x.split(" "))

In [4]:
dataset

,0
0,"[Far_Cry_6, God_of_War, Gran_Turismo_7, GTA_V,..."
1,"[UNCHARTED_4, God_of_War, FIFA_22, Dying_Light_2]"
2,"[Assassin's_Creed_Odyssey, God_of_War, UNCHART..."
3,"[DAYS_GONE, GTA_V, The_Last_of_Us_Part_II, God..."
4,"[Horizon_Forbidden_West, Gran_Turismo_7, GTA_V..."
...,...
39995,"[Dying_Light_2, The_Last_of_Us_Part_II, ARK:Su..."
39996,"[Assassin's_Creed_Odyssey, ARK:Survival_Evolve..."
39997,"[UNCHARTED_4, MAFIA_Trilogy, Dying_Light_2, Gh..."
39998,"[GTA_V, The_Last_of_Us_Part_II, It_Takes_Two, ..."


# Pass 1

In [5]:
SUPPORT_THRESHOLD = 100

In [6]:
support_unit = {}

for _, game_list in dataset.iterrows():
  for name in game_list[0]:
    if support_unit.get(name) is None:
      support_unit[name] = 0

    support_unit[name] += 1

In [7]:
frequent_unit = []

for game_name in support_unit:
  if support_unit[game_name] >= SUPPORT_THRESHOLD:
    frequent_unit.append(game_name)

frequent_unit = sorted(frequent_unit)

# Pass 2

In [8]:
support_pair = {}
for i, game_name1 in enumerate(frequent_unit):
  for j, game_name2 in enumerate(frequent_unit):
    if i < j:
        support_pair[(game_name1, game_name2)] = 0

In [9]:
for _, game_list in dataset.iterrows():
  game_list_names = sorted(game_list[0])

  for i, game_name1 in enumerate(game_list_names):
    for j, game_name2 in enumerate(game_list_names):
      if i<j and support_pair.get((game_name1, game_name2)) is not None:
        support_pair[(game_name1, game_name2)] += 1

In [10]:
frequent_pair = []

for pair in support_pair:
  if support_pair[pair] >= SUPPORT_THRESHOLD:
    frequent_pair.append(pair)

frequent_pair = sorted(frequent_pair)

In [11]:
def calculate_confidence_pair(left, right):
  return support_pair[tuple(sorted([right, left]))] / support_unit[left]

In [12]:
confidence_pair = []

for pair in frequent_pair:
  confidence_pair.append({"rule_left": pair[0],"rule_right": pair[1], "confidence": calculate_confidence_pair(left=pair[0], right=pair[1])})
  confidence_pair.append({"rule_left": pair[1],"rule_right": pair[0], "confidence": calculate_confidence_pair(left=pair[1], right=pair[0])})

In [13]:
sorted(confidence_pair, key=lambda item: (-item["confidence"], (item["rule_left"], item["rule_right"])))[:5]

[{'confidence': 0.9150213718705476,
  'rule_left': 'Dying_Light_2',
  'rule_right': 'The_Last_of_Us_Part_II'},
 {'confidence': 0.9136672282636442,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'The_Last_of_Us_Part_II'},
 {'confidence': 0.9136161739929545,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'GTA_V'},
 {'confidence': 0.9125440343084699,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'UNCHARTED_4'},
 {'confidence': 0.9119608941392128,
  'rule_left': 'Ghost_of_Tsushima',
  'rule_right': 'UNCHARTED_4'}]

# Pass 3

In [14]:
support_triple = {}

for pair in frequent_pair:
  for unit in frequent_unit:
    # Check not equality of unit and the two others
    if unit != pair[0] and unit != pair[1]:
      # Check order
      if unit < pair[0]:
        triple = (unit, pair[0], pair[1])
        # Check frequency of all pairs
        if (unit, pair[0]) in frequent_pair and (unit, pair[1]) in frequent_pair:
          support_triple[triple] = 0

In [15]:
for _, game_list in dataset.iterrows():
  game_list_names = sorted(game_list[0])

  for i, game_name1 in enumerate(game_list_names):
    for j, game_name2 in enumerate(game_list_names):
      for k, game_name3 in enumerate(game_list_names):
        if i<j<k and support_triple.get((game_name1, game_name2, game_name3)) is not None:
          support_triple[(game_name1, game_name2, game_name3)] += 1

In [16]:
frequent_triple = []

for triple in support_triple:
  if support_triple[triple] >= SUPPORT_THRESHOLD:
    frequent_triple.append(triple)

frequent_triple = sorted(frequent_triple)

In [17]:
def calculate_confidence_triple(left, right):
  return support_triple[tuple(sorted([left[0], left[1], right]))] / support_pair[tuple(sorted(left))]

In [18]:
confidence_triple = []

for triple in frequent_triple:
  confidence_triple.append({"rule_left": (triple[0], triple[1]), "rule_right": triple[2], "confidence": calculate_confidence_triple(left=(triple[0], triple[1]), right=triple[2])})
  confidence_triple.append({"rule_left": (triple[0], triple[2]), "rule_right": triple[1], "confidence": calculate_confidence_triple(left=(triple[0], triple[2]), right=triple[1])})
  confidence_triple.append({"rule_left": (triple[1], triple[2]), "rule_right": triple[0], "confidence": calculate_confidence_triple(left=(triple[1], triple[2]), right=triple[0])})

In [19]:
sorted(confidence_triple, key=lambda item: (-item["confidence"], (item["rule_left"], item["rule_right"])))[:5]

[{'confidence': 0.9635825196094125,
  'rule_left': ("Assassin's_Creed_Odyssey", 'DAYS_GONE'),
  'rule_right': 'The_Last_of_Us_Part_II'},
 {'confidence': 0.9634772836731418,
  'rule_left': ('A_Way_Out', 'Ghost_of_Tsushima'),
  'rule_right': 'UNCHARTED_4'},
 {'confidence': 0.9633978459794316,
  'rule_left': ('Far_Cry_6', 'Ghost_of_Tsushima'),
  'rule_right': 'GTA_V'},
 {'confidence': 0.9633881975110398,
  'rule_left': ('ARK:Survival_Evolved', 'Red_Dead_Redemption_2'),
  'rule_right': 'UNCHARTED_4'},
 {'confidence': 0.9633336001921383,
  'rule_left': ('Ghost_of_Tsushima', 'Gran_Turismo_7'),
  'rule_right': 'The_Last_of_Us_Part_II'}]